- 참고 : https://sieon-dev.tistory.com/15

## 1. 모듈 import (필요없는 모듈 삭제, 필요한 모듈 추가하면서 진행)
- pandas
- numpy
- matplotlib
- re
- konlpy
- tensorflow
- seaborn
- pandas_profiling

In [17]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tqdm import tqdm
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import pandas_profiling
import sklearn
from sklearn.model_selection import train_test_split
import collections
from collections import Counter
import exchange_calendars as ecals
import datetime
import time
from wordcloud import WordCloud
import matplotlib as mpl
import nltk

### 1-1. 파이썬 & 모듈 버전 확인 (필요없는 모듈 삭제, 필요한 모듈 추가하면서 진행)

In [18]:
print(" python version : ", sys.version, "\n", "-" * 100)
print(" pandas version : ", pd.__version__, "\n", "-" * 100)
print(" numpy version : ", np.__version__, "\n", "-" * 100)
print(" re version : ", re.__version__, "\n", "-" * 100)
print(" tensorflow version : ", tensorflow.__version__, "\n", "-" * 100)
print(" seaborn version : ", sns.__version__, "\n", "-" * 100)
print(" pandas_profiling version : ", pandas_profiling.__version__, "\n", "-" * 100)
print(" sklearn version : ", sklearn.__version__, "\n", "-" * 100)
print(" exhange_calendars version : ", ecals.__version__, "\n", "-" * 100)
print(" nltk version : ", nltk.__version__, "\n", "-" * 100)

 python version :  3.8.8 (tags/v3.8.8:024d805, Feb 19 2021, 13:18:16) [MSC v.1928 64 bit (AMD64)] 
 ----------------------------------------------------------------------------------------------------
 pandas version :  1.4.1 
 ----------------------------------------------------------------------------------------------------
 numpy version :  1.22.3 
 ----------------------------------------------------------------------------------------------------
 re version :  2.2.1 
 ----------------------------------------------------------------------------------------------------
 tensorflow version :  2.8.0 
 ----------------------------------------------------------------------------------------------------
 seaborn version :  0.11.2 
 ----------------------------------------------------------------------------------------------------
 pandas_profiling version :  3.1.0 
 ----------------------------------------------------------------------------------------------------
 sklearn version : 

### 1-2. 컴퓨터 사양 확인

![](2022-03-24-13-31-45.png)

## 2. 데이터 불러오기

In [27]:
# 뉴스데이터 불러오기
newsdata = pd.read_csv("C:\\Users\\easya\\Workspace\\HK_Project\\data\\Newsfile3.csv", encoding='euc-kr')
newsdata.tail(3)

,date,title
58439,20151019,"[특징주]제주반도체, 美·中 대규모 투자유치에 '上'"
58440,20151019,[반도체] 2016 반도체 투자 감소 예상 - 미래
58441,20151019,"제주반도체, '해외투자유치설' 조회공시 요구"


### 2-1. 휴장일 데이터 매칭

In [28]:
# newsdata의 datatype 확인
newsdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58442 entries, 0 to 58441
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    58442 non-null  int64 
 1   title   58442 non-null  object
dtypes: int64(1), object(1)
memory usage: 913.3+ KB


- type 변환

In [29]:
# datatype : int -> string
newsdata = newsdata.astype({'date':'string'})
newsdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58442 entries, 0 to 58441
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    58442 non-null  string
 1   title   58442 non-null  object
dtypes: object(1), string(1)
memory usage: 913.3+ KB


In [30]:
# datatype : string -> datetime
newsdata['date'] = pd.to_datetime(newsdata['date'])
newsdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58442 entries, 0 to 58441
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    58442 non-null  datetime64[ns]
 1   title   58442 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 913.3+ KB


- 뉴스데이터의 처음 날짜와 마지막 날짜 확인

In [31]:
first_date = newsdata['date'][0]
last_date = newsdata['date'][-1:]
print(first_date, last_date)

2022-03-16 00:00:00 58441   2015-10-19
Name: date, dtype: datetime64[ns]


In [32]:
# newsdata -> newsdata_all : 처음 날짜와 마지막 날짜가 포함된 데이터로 이름 변결
newsdata_all = newsdata[newsdata['date'].between('2015-10-19', '2022-03-16')]
newsdata_all

,date,title
0,2022-03-16,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자"
1,2022-03-16,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”
2,2022-03-16,"“韓, 세계 2위 中수입시장 비중 줄어…반도체 외 돌파구 필요”"
3,2022-03-16,"[특징주] 코리아써키트, 반도체 패키지기판 성장 기대감에 5%↑"
4,2022-03-16,"[특징주] 반도체 대장주 강세…SK하이닉스, 2% 넘게 상승"
...,...,...
58437,2015-10-19,세계 반도체 M&A 1006억달러
58438,2015-10-19,반도체社 인수·합병 봇물…올 들어 1000억달러 돌파
58439,2015-10-19,"[특징주]제주반도체, 美·中 대규모 투자유치에 '上'"
58440,2015-10-19,[반도체] 2016 반도체 투자 감소 예상 - 미래


- 리스트로 변환

In [33]:
# 'date' 컬럼 리스트로 변경
news = newsdata_all['date'].to_list()
news[:5]

[Timestamp('2022-03-16 00:00:00'),
 Timestamp('2022-03-16 00:00:00'),
 Timestamp('2022-03-16 00:00:00'),
 Timestamp('2022-03-16 00:00:00'),
 Timestamp('2022-03-16 00:00:00')]

### 2-2. 한국 거래소 개장일 확인

In [34]:
# 한국코드
XKRX = ecals.get_calendar("XKRX")       
# 2021-01-01은 개장일인지 확인 : 'False' -> 휴장일 , 'True' -> 개장일
print("2021-01-01 개장 확인: ", XKRX.is_session('2021-01-01'))    
# 오늘(22-03-24)은 개장일인지 확인 :'False' -> 휴장일 , 'True' -> 개장일
print("오늘날짜 개장일 확인: ", XKRX.is_session(datetime.date.today().strftime('%Y-%m-%d')))  
# 다음 개장일은 언제인지 확인
print("다음 개장일 날짜 확인: ", XKRX.next_open(pd.Timestamp.today().strftime('%Y-%m-%d')))

2021-01-01 개장 확인:  False
오늘날짜 개장일 확인:  False
다음 개장일 날짜 확인:  2022-03-28 00:00:00+00:00


In [35]:
# 뉴스데이터 날짜 다음날에 해당하는 개장일 날짜 확인
next_open_list = []

for day in news:
    next_open_date = XKRX.next_open(day)
    next_open_list.append(next_open_date)

In [36]:
next_open_list[:5]

[Timestamp('2022-03-17 00:00:00+0000', tz='UTC'),
 Timestamp('2022-03-17 00:00:00+0000', tz='UTC'),
 Timestamp('2022-03-17 00:00:00+0000', tz='UTC'),
 Timestamp('2022-03-17 00:00:00+0000', tz='UTC'),
 Timestamp('2022-03-17 00:00:00+0000', tz='UTC')]

In [37]:
# 뉴스나온 날짜 다음 개장일 컬럼 추가
newsdata_all['NextOpenDate'] = next_open_list
newsdata_all.head(3)

,date,title,NextOpenDate
0,2022-03-16,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자",2022-03-17 00:00:00+00:00
1,2022-03-16,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”,2022-03-17 00:00:00+00:00
2,2022-03-16,"“韓, 세계 2위 中수입시장 비중 줄어…반도체 외 돌파구 필요”",2022-03-17 00:00:00+00:00


In [38]:
newsdata_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 0 to 58441
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   date          58442 non-null  datetime64[ns]     
 1   title         58442 non-null  object             
 2   NextOpenDate  58442 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), object(1)
memory usage: 1.8+ MB


- type 변환

In [39]:
newsdata_all = newsdata_all.astype({'NextOpenDate':'string'})
newsdata_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 0 to 58441
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          58442 non-null  datetime64[ns]
 1   title         58442 non-null  object        
 2   NextOpenDate  58442 non-null  string        
dtypes: datetime64[ns](1), object(1), string(1)
memory usage: 1.8+ MB


In [40]:
a = newsdata_all['NextOpenDate'][0]
b = a.split(' ')[0]
b

'2022-03-17'

In [41]:
newsdata_all['NextOpenDate'] = newsdata_all['NextOpenDate'].apply(lambda x: x.split(' ')[0])
newsdata_all.head(3)

,date,title,NextOpenDate
0,2022-03-16,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자",2022-03-17
1,2022-03-16,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”,2022-03-17
2,2022-03-16,"“韓, 세계 2위 中수입시장 비중 줄어…반도체 외 돌파구 필요”",2022-03-17


In [42]:
newsdata_all['NextOpenDate'] = pd.to_datetime(newsdata_all['NextOpenDate'])
newsdata_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 0 to 58441
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          58442 non-null  datetime64[ns]
 1   title         58442 non-null  object        
 2   NextOpenDate  58442 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 3.8+ MB


In [43]:
# 개장일 기준 'Year' 컬럼 추가
newsdata_all['NextOpenDateYear'] = newsdata_all['NextOpenDate'].dt.year
newsdata_all.head(3)

,date,title,NextOpenDate,NextOpenDateYear
0,2022-03-16,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자",2022-03-17,2022
1,2022-03-16,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”,2022-03-17,2022
2,2022-03-16,"“韓, 세계 2위 中수입시장 비중 줄어…반도체 외 돌파구 필요”",2022-03-17,2022


In [44]:
# col의 인덱스 변경
col = newsdata_all.columns.to_numpy()
col

array(['date', 'title', 'NextOpenDate', 'NextOpenDateYear'], dtype=object)

In [45]:
col = col[[0,3,2,1]]
col

array(['date', 'NextOpenDateYear', 'NextOpenDate', 'title'], dtype=object)

In [46]:
newsdata_all = newsdata_all[col]
newsdata_all.head(3)

,date,NextOpenDateYear,NextOpenDate,title
0,2022-03-16,2022,2022-03-17,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자"
1,2022-03-16,2022,2022-03-17,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”
2,2022-03-16,2022,2022-03-17,"“韓, 세계 2위 中수입시장 비중 줄어…반도체 외 돌파구 필요”"


## 3. 데이터 확인

In [47]:
# 데이터 갯수 : 58442개
# 컬럼 갯수 : 4개
newsdata_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 0 to 58441
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              58442 non-null  datetime64[ns]
 1   NextOpenDateYear  58442 non-null  int64         
 2   NextOpenDate      58442 non-null  datetime64[ns]
 3   title             58442 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 4.2+ MB


- 결측치 확인

In [48]:
# 결측치 확인
newsdata_all.isnull().sum()

date                0
NextOpenDateYear    0
NextOpenDate        0
title               0
dtype: int64

## 4. 기업별 '현재가', '당일종가-어제종가', '상승률', 'label' 데이터와 뉴스데이터 병합

### 4-1. 반도체 기업 데이터 불러오기 : 005290

In [59]:
df1 = pd.read_excel('C:\\Users\\easya\\Workspace\\HK_Project\\data\\반도체주가데이터\\반도체주가데이터\\덕산네오룩스(213420).xlsx')
df1.head(2)

,Unnamed: 0.1,Unnamed: 0,일자,현재가,당일종가-어제종가,상승률,label
0,0,0,20220314,39050,-1550,-0.038177,0
1,1,1,20220311,40600,-550,-0.013366,0


In [60]:
df1.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
df1.head(2)

,일자,현재가,당일종가-어제종가,상승률,label
0,20220314,39050,-1550,-0.038177,0
1,20220311,40600,-550,-0.013366,0


- 긍정 & 부정 라벨링
    - 상승률이 0보다 크면 `1`로 매칭
    - 상승률이 0보다 작으면 `0`으로 매칭

In [61]:
df1['label'] = df1.apply(lambda x: 1 if x['상승률'] >= 0 else 0, axis=1)
df1

,일자,현재가,당일종가-어제종가,상승률,label
0,20220314,39050,-1550,-0.038177,0
1,20220311,40600,-550,-0.013366,0
2,20220310,41150,1150,0.028750,1
3,20220308,40000,100,0.002506,1
4,20220307,39900,-1300,-0.031553,0
...,...,...,...,...,...
1739,20150212,11774,-526,-0.042764,0
1740,20150211,12300,176,0.014517,1
1741,20150210,12124,-852,-0.065660,0
1742,20150209,12976,150,0.011695,1


- 라벨링 분포 확인

In [62]:
plt.savefig('test.png')
sns.countplot(x='label', data=df1)

<AxesSubplot:xlabel='label', ylabel='count'>

- data type 변경

In [63]:
# data type : int -> string
df1 = df1.astype({'일자':'string'})
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1744 entries, 0 to 1743
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   일자         1744 non-null   string 
 1   현재가        1744 non-null   int64  
 2   당일종가-어제종가  1744 non-null   int64  
 3   상승률        1744 non-null   float64
 4   label      1744 non-null   int64  
dtypes: float64(1), int64(3), string(1)
memory usage: 68.2 KB


In [64]:
# data type : string -> datetime
df1['일자'] = pd.to_datetime(df1['일자'])
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1744 entries, 0 to 1743
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   일자         1744 non-null   datetime64[ns]
 1   현재가        1744 non-null   int64         
 2   당일종가-어제종가  1744 non-null   int64         
 3   상승률        1744 non-null   float64       
 4   label      1744 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 68.2 KB


### 4-2. 뉴스데이터 컬럼명 변경

In [65]:
# 덕산네오룩스의 데이터프레임과 병합하기 위해 컬럼명 변경
newsdata_all.rename(columns={'NextOpenDate':'일자'}, inplace=True)
newsdata_all.head(2)

,date,NextOpenDateYear,일자,title
0,2022-03-16,2022,2022-03-17,"유럽 반도체 공급망 지원 호응… 인텔, 10년간 110조 통큰 투자"
1,2022-03-16,2022,2022-03-17,반도체 또다시 ‘쩐의 전쟁‘…인텔 “유럽 전역에 110조 투자”


### 4-3. 뉴스데이터와 덕산네오룩스 주가데이터 병합

In [66]:
merge_data = pd.merge(newsdata_all, df1, on='일자', how='inner')
merge_data.head(2)

,date,NextOpenDateYear,일자,title,현재가,당일종가-어제종가,상승률,label
0,2022-03-13,2022,2022-03-14,전함 포템킨·오데사·네온사인…반도체는 왜 또 위기일까[인더독],39050,-1550,-0.038177,0
1,2022-03-13,2022,2022-03-14,"한국기업, 반도체·전자 부품 조달 ‘빨간불’",39050,-1550,-0.038177,0


- '일자', 'title', 'label'만 남긴 나머지 컬럼 삭제

In [67]:
merge_data = merge_data[['일자', 'title', 'label']]
merge_data.head(5)

,일자,title,label
0,2022-03-14,전함 포템킨·오데사·네온사인…반도체는 왜 또 위기일까[인더독],0
1,2022-03-14,"한국기업, 반도체·전자 부품 조달 ‘빨간불’",0
2,2022-03-14,“삼성·SK만 잘하면 뭐해?” 한국 반도체 진짜 ‘약점’이 바로 [비즈360],0
3,2022-03-14,"""반도체 일본꼴 날 수도…수도권 대학 반도체학과 정원 풀어야"" [당선인에 바란다]",0
4,2022-03-14,"[우크라 침공] 러시아, 반도체소자 등 500개 품목 대외수출 금지·제한",0


## 5. 데이터 전처리
    - 한글 이외의 문자열은 빈칸(`''`)처리
    - 품사 중 명사만 추출하여 'nouns' 컬럼에 삽입

In [68]:
okt = Okt()
n_ = []
title_rename = []

for i in range(len(merge_data)):
    if (i % 10000 == 0):
        print(i, "단계 완료")
    title_rename.append(re.sub("[\(\[].*?[\)\]]", "", merge_data.iloc[i]['title']))
    n_.append(' '.join(okt.nouns(merge_data.iloc[i]['title'])))
merge_data['nouns'] = n_
merge_data['title'] = title_rename
merge_data = merge_data[merge_data['nouns'] != '']

# 2차 불용어 제거
merge_data['title'] = merge_data['title'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]','')
merge_data['title'].replace('', np.nan, inplace=True)
merge_data = merge_data.dropna(how='any')

0 단계 완료
10000 단계 완료
20000 단계 완료
30000 단계 완료
40000 단계 완료
50000 단계 완료


In [69]:
merge_data.head(3)

,일자,title,label,nouns
0,2022-03-14,전함포템킨오데사네온사인반도체는왜또위기일까,0,전함 포템킨 오데사 네온사인 반도체 왜 또 위기 인 더독
1,2022-03-14,한국기업반도체전자부품조달빨간불,0,한국 기업 반도체 전자 부품 조달 불
2,2022-03-14,삼성만잘하면뭐해한국반도체진짜약점이바로,0,삼성 만 뭐 한국 반도체 진짜 약점 이 바로 비즈


In [70]:
news_labeling = merge_data.copy()

In [71]:
# 결측치 확인
news_labeling.dropna(how='any', inplace=True)
news_labeling.shape

(58343, 4)

- label 분포 확인

In [72]:
sns.countplot(x='label', data=news_labeling)

<AxesSubplot:xlabel='label', ylabel='count'>

In [73]:
# 라벨링 값 개수 확인
news_labeling['label'].value_counts()

1    29953
0    28390
Name: label, dtype: int64

## 6. train_data, test_data 분할

In [74]:
# train, test set 분리
train_data, test_data = train_test_split(news_labeling)

### 6-1. train_data

In [75]:
# train_data 확인
train_data.head(2)

,일자,title,label,nouns
23893,2020-04-21,삼성전자위기를이길힘은혁신뿐로봇반도체초격차전략,0,삼성 전자 위기 힘 혁신 로봇 반도체 초 격차 전략
16323,2021-02-24,현대차기아도반도체확보경고등단위로생산조절,0,현대차 기아 반도체 확보 고등 단위 생산 조절


In [76]:
# train_data shape 확인
train_data.shape

(43757, 4)

### 6-2. test_data

In [77]:
# test_data 확인
test_data.head(2)

,일자,title,label,nouns
11877,2021-05-14,차반도체회의앞반도체종합전략발표더과감한세제혜택아쉽다,1,차 반도체 회의 앞 반도체 종합 전략 발표 더 과감 세제 혜택
8560,2021-08-04,하이닉스외인매수반도체지수호조에강세,0,특징 주 하이닉스 외인 매수 반도체 지수 호조 강세


In [78]:
# test_data shape 확인
test_data.shape

(14586, 4)

## 7. 감성사전 구축

- 한 글자인 단어는 제외. 두 글자 이상인 단어들의 점수 초기화 진행

In [79]:
vocab = {}
count = 0
for i in news_labeling['nouns']:
    i = i.split(' ')
    # news_labeling에 있는 데이터의 양만큼 순회
    for j in range(len(i)):
        # i 번째 행의 리스트에 있는 j번째 단어의 길이가 1 이하일 경우 count에 1 추가
        if i[j] in vocab or len(i[j]) <= 1:
            count += 1
            pass
        else:
            # i 번째 행의 리스트에 있는 j번째 단어의 길이가 1 이상일 경우 0
            vocab[i[j]] = 0

In [80]:
vocab

{'전함': 0,
 '포템킨': 0,
 '오데사': 0,
 '네온사인': 0,
 '반도체': 0,
 '위기': 0,
 '더독': 0,
 '한국': 0,
 '기업': 0,
 '전자': 0,
 '부품': 0,
 '조달': 0,
 '삼성': 0,
 '진짜': 0,
 '약점': 0,
 '바로': 0,
 '비즈': 0,
 '일본': 0,
 '수도': 0,
 '수도권': 0,
 '대학': 0,
 '학과': 0,
 '정원': 0,
 '당선인': 0,
 '침공': 0,
 '러시아': 0,
 '반도체소자': 0,
 '품목': 0,
 '대외': 0,
 '수출': 0,
 '금지': 0,
 '제한': 0,
 '호국': 0,
 '통제': 0,
 '공급': 0,
 '우려': 0,
 '애플': 0,
 '수혜': 0,
 '가능성': 0,
 '전기차': 0,
 '원가': 0,
 '상승': 0,
 '비상': 0,
 '배터리': 0,
 '철강': 0,
 '고공': 0,
 '행진': 0,
 '경제': 0,
 '제재': 0,
 '보복': 0,
 '시작': 0,
 '소자': 0,
 '속보': 0,
 '발표': 0,
 '포함': 0,
 '전쟁': 0,
 '가스': 0,
 '생산': 0,
 '중단': 0,
 '네온': 0,
 '절반': 0,
 '대란': 0,
 '심화': 0,
 '파문': 0,
 '대전': 0,
 '조짐': 0,
 '격화': 0,
 '인텔': 0,
 '계속': 0,
 '윤석열': 0,
 '공약': 0,
 '실현': 0,
 '집적회로': 0,
 '상보': 0,
 '산다': 0,
 '두산': 0,
 '통큰': 0,
 '강행': 0,
 '이유': 0,
 '브로드컴': 0,
 '주주': 0,
 '환원': 0,
 '기대': 0,
 '조치': 0,
 '여파': 0,
 '변신': 0,
 '승부': 0,
 '과연': 0,
 '자등': 0,
 '인력': 0,
 '양성': 0,
 '기금': 0,
 '조성': 0,
 '도로': 0,
 '지수': 0,
 '하락': 0,
 '악재': 0,
 '코스피': 0,


### 7-1. train_data

- 상승비율과 하락비율을 정의해준 다음 라벨 값이 1이면 하락 비율을 각 단어에 더해주고 라벨값이 0이면 상승 비율을 차감
- 라벨값이 1이면 상승인데 왜 상승 비율을 더해주지 않았을지 : 라벨값이 1인 데이터가 아닌 데이터보다 훨씬 많다면 해당 단어들의 점수가 너무 커져서 점수가 고르지 못한 감성사전이 만들어지기 때문에 정규화했기 때문

In [81]:
up = 29202
down = 29141

up_ratio = up/(up+down)
down_ratio = down/(up+down)

for i,w in enumerate(train_data['nouns']):
    w = w.split(' ')
    if (train_data.iloc[i]['label']==1):
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
              continue
            vocab[noun] = vocab[noun] + down_ratio
    else:
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
              continue
            vocab[noun] = vocab[noun] - up_ratio

In [82]:
# 감성 점수 확인
vocab

{'전함': 0,
 '포템킨': 0,
 '오데사': 0,
 '네온사인': 0,
 '반도체': 674.4680081583715,
 '위기': -6.739951665152669,
 '더독': -0.00104554102463017,
 '한국': 41.423906895430285,
 '기업': 24.649452376462857,
 '전자': 66.04396414308736,
 '부품': 16.207771283615784,
 '조달': 16.96445160516256,
 '삼성': -28.029052328471785,
 '진짜': -7.018819738443348,
 '약점': -2.5036593935862053,
 '바로': -0.5036593935862056,
 '비즈': -5.038685017911316,
 '일본': 24.26318495792114,
 '수도': 3.9529506538916306,
 '수도권': -3.5203880499802884,
 '대학': 6.981180261556655,
 '학과': -8.535025624325128,
 '정원': 0.9968633769261095,
 '당선인': -1.0010455410246302,
 '침공': 0.9926812128275898,
 '러시아': 1.9790891795073975,
 '반도체소자': -6.506796016660098,
 '품목': 2.8954458975369826,
 '대외': -3.019865279467971,
 '수출': 212.67576230226572,
 '금지': -9.03241177176354,
 '제한': -7.0637780025024615,
 '호국': -0.5005227705123151,
 '통제': 2.4911129012906446,
 '공급': 70.4735101040432,
 '우려': -17.29170594587198,
 '애플': 18.38133109370442,
 '수혜': -18.690811236995103,
 '가능성': 2.9153111770049636,
 '

- 뉴스의 평균 감성 점수 산출 뒤 컬럼에 추가

In [83]:
total = []
for i,w  in enumerate(train_data['nouns']):
    sent_score = 0
    w= w.split(' ')
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab):
          continue
        else:
          sent_score = sent_score + vocab[j]
    total.append(sent_score/len(w))
train_data['sent_score'] = total

- sent_score가 감성사전의 평균 점수보다 높으면 1, 낮으면 0으로 새롭게 라벨링

In [84]:
sum = 0
for i in range(len(vocab)):
  sum = sum + list(vocab.values())[i]
sent_mean = sum/len(vocab)

a_ = []
for i in range(len(train_data)):
  if(train_data.iloc[i]['sent_score']>sent_mean):
    a_.append(1)
  else:
    a_.append(0)
train_data['sent_label'] = a_

In [85]:
train_data.head()

,일자,title,label,nouns,sent_score,sent_label
23893,2020-04-21,삼성전자위기를이길힘은혁신뿐로봇반도체초격차전략,0,삼성 전자 위기 힘 혁신 로봇 반도체 초 격차 전략,70.862349,1
16323,2021-02-24,현대차기아도반도체확보경고등단위로생산조절,0,현대차 기아 반도체 확보 고등 단위 생산 조절,94.196853,1
45657,2018-01-23,반도체정유화학호실적에성과급도두둑최대연봉지급,0,반도체 정유 화학 호실 성과급 두둑 최대 연봉 지급,93.981453,1
3087,2021-12-16,영창케미칼반도체장비세대경영에도앞장,0,영창 케미칼 반도체 장비 세대 경영 앞장,102.919652,1
25240,2020-02-05,산업계강타한신종코로나악재항공이어반도체배터리까지,0,산업 강타 신종 코로나 악재 항공 반도체 배터리,85.052380,1


### 7-2. test_data
- train_data와 동일한 방식으로 진행

In [86]:
up = 29202
down = 29141

up_ratio = up/(up+down)
down_ratio = down/(up+down)

for i,w in enumerate(test_data['nouns']):
    w = w.split(' ')
    if (test_data.iloc[i]['label']==1):
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
              continue
            vocab[noun] = vocab[noun] + down_ratio
    else:
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
              continue
            vocab[noun] = vocab[noun] - up_ratio

In [87]:
# 감성 점수 확인
vocab

{'전함': -0.5005227705123151,
 '포템킨': -0.5005227705123151,
 '오데사': -0.5005227705123151,
 '네온사인': -0.5005227705123151,
 '반도체': 822.6097218165519,
 '위기': 4.680064446463094,
 '더독': -0.00209108204926034,
 '한국': 46.7393689045842,
 '기업': 45.37447508698603,
 '전자': 109.86563940832087,
 '부품': 34.099035017054305,
 '조달': 19.958178359014774,
 '삼성': -24.698781344801173,
 '진짜': -7.019865279467979,
 '약점': -3.0041821640985202,
 '바로': -0.00522770512315085,
 '비즈': -5.54966319866994,
 '일본': 33.6915653977341,
 '수도': 6.93622199749754,
 '수도권': -7.027184066640388,
 '대학': 10.97386147438424,
 '학과': -9.547572116620698,
 '정원': 0.9958178359014792,
 '당선인': -1.5015683115369454,
 '침공': 2.9895445897536996,
 '러시아': 3.9686337692610976,
 '반도체소자': -9.009409869221674,
 '품목': 3.3646024373103867,
 '대외': -8.029275148689647,
 '수출': 265.37958281202555,
 '금지': -11.543389952522164,
 '제한': -8.08782544606897,
 '호국': -1.0010455410246302,
 '통제': 3.485885196167495,
 '공급': 102.9361020173793,
 '우려': -14.387895720138015,
 '애플': 11.3499648

- 뉴스의 평균 감성 점수 산출 뒤 컬럼에 추가

In [88]:
total = []
for i,w  in enumerate(test_data['nouns']):
    sent_score = 0
    w= w.split(' ')
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab):
          continue
        else:
          sent_score = sent_score + vocab[j]
    total.append(sent_score/len(w))
test_data['sent_score'] = total

- sent_score가 감성사전의 평균 점수보다 높으면 1, 낮으면 0으로 새롭게 라벨링

In [89]:
sum = 0
for i in range(len(vocab)):
  sum = sum + list(vocab.values())[i]
sent_mean = sum/len(vocab)

a_ = []
for i in range(len(test_data)):
  if(test_data.iloc[i]['sent_score']>sent_mean):
    a_.append(1)
  else:
    a_.append(0)
test_data['sent_label'] = a_

In [90]:
test_data.head()

,일자,title,label,nouns,sent_score,sent_label
11877,2021-05-14,차반도체회의앞반도체종합전략발표더과감한세제혜택아쉽다,1,차 반도체 회의 앞 반도체 종합 전략 발표 더 과감 세제 혜택,138.505755,1
8560,2021-08-04,하이닉스외인매수반도체지수호조에강세,0,특징 주 하이닉스 외인 매수 반도체 지수 호조 강세,87.437647,1
22089,2020-07-01,수출규제년반도체디스플레이소부장경쟁력상승,0,수출 규제 반도체 디스플레이 부장 경쟁력 상승,163.851117,1
22144,2020-06-30,한컴글로벌반도체설계자산회사과파트너십,1,한컴 글로벌 반도체 설계 자산 회사 과 파트너,92.947352,1
19850,2020-10-21,서울반도체자연광조명썬라이크유럽유치원에적용,0,서울 반도체 자연 광 조명 이크 유럽 유치원 적용,96.125354,1


## 8. 워드 클라우드

### 8-1. train_data

In [91]:
# train_data > nouns 컬럼 > 0번째 데이터의 단어 출력
train_data.nouns.to_list()[0]

'삼성 전자 위기 힘 혁신 로봇 반도체 초 격차 전략'

In [92]:
# train_data의 길이 확인
len(train_data.nouns.to_list())

43757

- train_data의 'nouns' 데이터 합치기

In [93]:
# 'nouns' 컬럼 데이터 리스트로 변환
nouns_list_train = []

for i in range(0, len(train_data.nouns.to_list())):
    noun = train_data.nouns.to_list()[i]
    nouns_list_train.append(noun)

In [94]:
nouns_list_train[:5]

['삼성 전자 위기 힘 혁신 로봇 반도체 초 격차 전략',
 '현대차 기아 반도체 확보 고등 단위 생산 조절',
 '반도체 정유 화학 호실 성과급 두둑 최대 연봉 지급',
 '영창 케미칼 반도체 장비 세대 경영 앞장',
 '산업 강타 신종 코로나 악재 항공 반도체 배터리']

In [95]:
nouns_list_train[0]

'삼성 전자 위기 힘 혁신 로봇 반도체 초 격차 전략'

In [96]:
nouns_list_train[0].split() + nouns_list_train[1].split()

['삼성',
 '전자',
 '위기',
 '힘',
 '혁신',
 '로봇',
 '반도체',
 '초',
 '격차',
 '전략',
 '현대차',
 '기아',
 '반도체',
 '확보',
 '고등',
 '단위',
 '생산',
 '조절']

In [97]:
# WordCloud에 사용할 리스트 생성
nouns_total_train = []

for i in range(0, len(nouns_list_train)):
    nouns_total_train += nouns_list_train[i].split()

In [98]:
nouns_total_train

['삼성',
 '전자',
 '위기',
 '힘',
 '혁신',
 '로봇',
 '반도체',
 '초',
 '격차',
 '전략',
 '현대차',
 '기아',
 '반도체',
 '확보',
 '고등',
 '단위',
 '생산',
 '조절',
 '반도체',
 '정유',
 '화학',
 '호실',
 '성과급',
 '두둑',
 '최대',
 '연봉',
 '지급',
 '영창',
 '케미칼',
 '반도체',
 '장비',
 '세대',
 '경영',
 '앞장',
 '산업',
 '강타',
 '신종',
 '코로나',
 '악재',
 '항공',
 '반도체',
 '배터리',
 '용인',
 '반도체',
 '클러스터',
 '등',
 '맞춤',
 '지원',
 '소재',
 '부품',
 '장비',
 '경쟁력',
 '강화',
 '생생',
 '코스닥',
 '제주',
 '반도체',
 '자',
 '사주',
 '취득',
 '결정',
 '하이닉스',
 '충북',
 '청주',
 '신규',
 '반도체',
 '공장',
 '투자',
 '반도체',
 '선방',
 '수출',
 '뚝',
 '삼성',
 '전자',
 '차세대',
 '반도체',
 '세포',
 '치료',
 '제',
 '양자',
 '컴퓨팅',
 '분야',
 '연구',
 '지원',
 '종합',
 '면세점',
 '구매',
 '한도',
 '폐지',
 '반도체',
 '탄소',
 '중립',
 '분야',
 '세제',
 '혜택',
 '강화',
 '이재용',
 '첫',
 '전국',
 '장경영',
 '온양',
 '반도체',
 '공정',
 '직접',
 '한미',
 '반도체',
 '규모',
 '반도체',
 '제조',
 '장비',
 '수출',
 '하이닉스',
 '를',
 '반도체',
 '투자',
 '시기',
 '반도체',
 '부진',
 '제조업',
 '수',
 '투자',
 '최저',
 '자본',
 '재',
 '역대',
 '최',
 '대폭',
 '세계',
 '반도체',
 '매출',
 '램값',
 '서울',
 '반도체',
 '하반기',
 '수요',
 '불확실',
 '목표',
 '금투',
 '엠',


In [99]:
# 가장 많이 나온 단어 저장
counts = Counter(nouns_total_train)
tags = counts.most_common(30000)
tags

[('반도체', 45014),
 ('삼성', 9620),
 ('전자', 6611),
 ('수출', 4446),
 ('장비', 3565),
 ('투자', 3411),
 ('공급', 2920),
 ('하이닉스', 2712),
 ('계약', 2263),
 ('규모', 2157),
 ('실적', 1837),
 ('공장', 1814),
 ('주', 1757),
 ('영업', 1693),
 ('서울', 1662),
 ('기업', 1627),
 ('산업', 1599),
 ('등', 1550),
 ('한미', 1502),
 ('생산', 1440),
 ('최대', 1393),
 ('기술', 1336),
 ('시스템', 1328),
 ('위', 1292),
 ('용', 1268),
 ('매출', 1249),
 ('제조', 1240),
 ('전망', 1236),
 ('사업', 1233),
 ('시장', 1229),
 ('익', 1152),
 ('증가', 1148),
 ('종합', 1136),
 ('한국', 1102),
 ('세계', 1079),
 ('내년', 1072),
 ('소재', 1061),
 ('상승', 1026),
 ('올해', 1002),
 ('중국', 998),
 ('감소', 990),
 ('개월', 982),
 ('디스플레이', 978),
 ('부진', 957),
 ('부족', 946),
 ('호황', 925),
 ('글로벌', 908),
 ('업계', 904),
 ('성장', 902),
 ('하락', 900),
 ('메모리', 894),
 ('이재용', 884),
 ('이익', 866),
 ('정부', 839),
 ('특징', 823),
 ('업체', 777),
 ('차', 759),
 ('자동차', 758),
 ('코스닥', 751),
 ('지원', 749),
 ('현대차', 743),
 ('인텔', 732),
 ('개발', 724),
 ('검사', 723),
 ('뉴스', 701),
 ('협력', 694),
 ('회복', 666),
 ('미래', 657),
 

- 한글 폰트 사용

In [100]:
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'

- 필요없는 단어 제거 : stopwords 설정

In [101]:
# 불용어 지정
stopwords = "램 사 액 첫 제 세 달 내년 반도체 분기 억 월 차 년 중 주 일 등 조 억원 용 익 위 개월 이재용 것 조원 초"
stop_list = stopwords.split()

# stop_list에 없는 단어만 추출 > 리스트를 만들 것
words_list = [word for word in nouns_total_train if word not in stop_list]

- 단어 빈도수 살펴보기

In [102]:
words = nltk.Text(words_list)
plt.figure(figsize=(20,12))
# 많이 사용된 단어 보여주기
words.plot(100)        
plt.show()

In [103]:
# 빈도수가 200 이상인 단어 10개
data = words.vocab().most_common(200)
data[:10]

[('삼성', 9620),
 ('전자', 6611),
 ('수출', 4446),
 ('장비', 3565),
 ('투자', 3411),
 ('공급', 2920),
 ('하이닉스', 2712),
 ('계약', 2263),
 ('규모', 2157),
 ('실적', 1837)]

- train_data 워드클라우드 생성

In [104]:
wc = WordCloud(
    font_path='c:/Windows/Fonts/malgun.ttf',
    relative_scaling=0.2, background_color='white',
    colormap='coolwarm'
).generate_from_frequencies(dict(data))


In [105]:
plt.figure(figsize=(12,6))
plt.imshow(wc)
plt.axis('off')
plt.show()

### 8-2. test_data

In [106]:
# test_data > nouns 컬럼 > 0번째 데이터의 단어
test_data.nouns.to_list()[0]


# 'nouns' 컬럼 데이터 리스트로 변환
nouns_list_test = []

for i in range(0, len(test_data.nouns.to_list())):
    noun = test_data.nouns.to_list()[i]
    nouns_list_test.append(noun)


# 'nouns' 컬럼 데이터 단어 데이터 합치기
nouns_total_test = []

for i in range(0, len(nouns_list_test)):
    nouns_total_test += nouns_list_test[i].split()

# 가장 많이 나온 단어 저장
counts = Counter(nouns_total_test)
tags = counts.most_common(30000)
tags


# 불용어 지정
stopwords = "올해 작년 램 사 액 첫 제 세 달 내년 반도체 분기 억 월 차 년 중 주 일 등 조 억원 용 익 위 개월 이재용 것 조원 초"
stop_list = stopwords.split()

# stop_list에 없는 단어만 추출 > 리스트를 만들 것
words_list = [word for word in nouns_total_test if word not in stop_list]

words = nltk.Text(words_list)
plt.figure(figsize=(20,12))
# 많이 사용된 단어 보여주기
words.plot(100)        
plt.show()

In [107]:
# 빈도수가 200 이상인 단어 10개
data = words.vocab().most_common(200)
data[:10]

[('삼성', 3194),
 ('전자', 2254),
 ('수출', 1523),
 ('장비', 1210),
 ('투자', 1036),
 ('공급', 1028),
 ('하이닉스', 917),
 ('규모', 775),
 ('계약', 774),
 ('실적', 617)]

- test_data 워드클라우드 생성

In [108]:
wc = WordCloud(
    font_path='c:/Windows/Fonts/malgun.ttf',
    relative_scaling=0.2, background_color='white',
    colormap='coolwarm'
).generate_from_frequencies(dict(data))

plt.figure(figsize=(12,6))
plt.imshow(wc)
plt.axis('off')
plt.show()

## 9. 모델링

In [144]:
from konlpy.tag import Mecab

In [148]:
train_set, test_set = train_test_split(train_data, test_size=0.2, random_state= 42)
train_set.dropna(how='any', inplace=True)
test_set.dropna(how='any', inplace=True)
mecab = Mecab()
train_set['tokenized'] = test_set['title'].apply(mecab.morphs)
X_train = train_set['tokenized'].values
Y_train = train_set['sent_label'].values
X_val = test_set['tokenized'].values
Y_val = test_set['sent_label'].values

Exception: The MeCab dictionary does not exist at "/usr/local/lib/mecab/dic/mecab-ko-dic". Is the dictionary correctly installed?
You can also try entering the dictionary path when initializing the Mecab class: "Mecab('/some/dic/path')"